In [2]:
import sys
sys.path.append('../')
from src.data_processor import DataProcessor

In [ ]:
engine = DataProcessor(raw_data='../data/raw', processed_data='../data/processed/')
engine.load_data()
df = engine.merge_data()
print(df.head())

Data loaded successfully.
Error merging data: 'customer_id'


KeyError: 'customer_id'